This notebook is an integration to the Precipitation harvesting notebook. It illustrates how the missing values are estimated.

We construct the dataset consisting of different quadkeys each with its best approximation.

In [1]:
import pandas as pd
import os

cart = "C:\\Users\\Luca\\Downloads\\RWI\\precipitation"

df = pd.read_csv(f'{cart}\\precipitation_utili14.csv')
df

,lat,lon,precipitation_year,quadkey
0,-4.6,-81.2,1.154,21000132030002
1,-4.5,-81.2,1.230,21000132012202
2,-4.4,-81.2,1.344,21000132012000
3,-4.3,-81.2,1.406,21000132010022
4,-5.1,-81.1,1.116,21000132212101
...,...,...,...,...
130860,-8.6,127.0,3.986,31011230012011
130861,-8.5,127.0,4.002,31011230010211
130862,-8.5,127.1,3.946,31011230011200
130863,-8.4,127.2,2.710,31011212233323


In [2]:
df = df[['quadkey', 'precipitation_year']]
df

,quadkey,precipitation_year
0,21000132030002,1.154
1,21000132012202,1.230
2,21000132012000,1.344
3,21000132010022,1.406
4,21000132212101,1.116
...,...,...
130860,31011230012011,3.986
130861,31011230010211,4.002
130862,31011230011200,3.946
130863,31011212233323,2.710


In [3]:
df.quadkey.value_counts()

quadkey
21000132030002    1
30013210113200    1
30013030331020    1
30013030331222    1
30013030333200    1
                 ..
12223033203230    1
12223033221032    1
12223211003012    1
12223211221232    1
31011212322223    1
Name: count, Length: 130865, dtype: int64

In [4]:
for file in os.listdir(cart):
    if ('utili' in file) and ('14' not in file):
        temp = pd.read_csv(f'{cart}\\{file}')
        nome_nuovo = f'precipitation_year{file[-6:-4]}'
        temp = temp.rename(columns = {'precipitation_year': nome_nuovo})
        df = pd.concat([df, temp[['quadkey', nome_nuovo]]], ignore_index = True)

In [5]:
df

,quadkey,precipitation_year,precipitation_year11,precipitation_year12,precipitation_year13
0,21000132030002,1.154,NaN,NaN,NaN
1,21000132012202,1.230,NaN,NaN,NaN
2,21000132012000,1.344,NaN,NaN,NaN
3,21000132010022,1.406,NaN,NaN,NaN
4,21000132212101,1.116,NaN,NaN,NaN
...,...,...,...,...,...
5480067,12313231213101,NaN,NaN,NaN,2.870
5480068,12312232132331,NaN,NaN,NaN,3.346
5480069,30031002020333,NaN,NaN,NaN,3.102
5480070,12312211021323,NaN,NaN,NaN,5.150


We can see that there will still be around 18% of missing values in the dataset. It's however a loss we could accept since they can be later imputed in the final model.

In [6]:
with open('quad_paesi2.txt', 'r') as file:
    list_quads = file.read()
    list_quads = list_quads.split(',')

In [7]:
len(df.quadkey.unique()) / len(list_quads)

0.8163975537733065

In [8]:
df = df.astype({'quadkey': str})
len(df[df.quadkey.isin(list_quads)].quadkey.unique())

3134329

We create the dataset of final estimates containing all the quadkeys we will use in the study. First we assign to each quadkey 14 tile with a proper value its estimate.

In [9]:
presenti = list(df[df.quadkey.isin(list_quads)].quadkey.unique())

In [37]:
stime = pd.DataFrame(presenti, columns = ['quadkey'])
stime

,quadkey
0,21000132030002
1,21000132012202
2,21000132012000
3,21000132010022
4,21000132212101
...,...
3134324,12200210101123
3134325,21001101111122
3134326,21003102011232
3134327,30012330211323


In [38]:
stime = stime.merge(df.loc[df.precipitation_year.notnull(),['quadkey', 'precipitation_year']], on = 'quadkey', how = 'left')
stime

,quadkey,precipitation_year
0,21000132030002,1.154
1,21000132012202,1.230
2,21000132012000,1.344
3,21000132010022,1.406
4,21000132212101,1.116
...,...,...
3134324,12200210101123,NaN
3134325,21001101111122,NaN
3134326,21003102011232,NaN
3134327,30012330211323,NaN


We add an approximation variable representing the degree of approximation in our estimate. In the case of quadkey 14 values, there's no approximation as we just matched quadkeys across datasets.

In [40]:
import numpy as np

stime['approx'] = np.where(stime.precipitation_year.notnull(), 0, np.nan)
stime

,quadkey,precipitation_year,approx
0,21000132030002,1.154,0.0
1,21000132012202,1.230,0.0
2,21000132012000,1.344,0.0
3,21000132010022,1.406,0.0
4,21000132212101,1.116,0.0
...,...,...,...
3134324,12200210101123,NaN,NaN
3134325,21001101111122,NaN,NaN
3134326,21003102011232,NaN,NaN
3134327,30012330211323,NaN,NaN


In [41]:
stime.describe()

,precipitation_year,approx
count,130865.000000,130865.0
mean,2.764508,0.0
std,1.622454,0.0
min,0.000000,0.0
25%,1.710000,0.0
50%,2.560000,0.0
75%,3.480000,0.0
max,16.440001,0.0


We now add the column containing the estimates of quadkey 13 tiles. We fill the missing quadkey 14 tiles precipitation values with their quadkey 13 estimate, if there is one. The approximation for the quadkeys filled by their 13 estimate is thus 1.

In [42]:
stime = stime.merge(df.loc[df.precipitation_year13.notnull(),['quadkey', 'precipitation_year13']], on = 'quadkey', how = 'left')
stime

,quadkey,precipitation_year,approx,precipitation_year13
0,21000132030002,1.154,0.0,NaN
1,21000132012202,1.230,0.0,NaN
2,21000132012000,1.344,0.0,NaN
3,21000132010022,1.406,0.0,NaN
4,21000132212101,1.116,0.0,NaN
...,...,...,...,...
3134324,12200210101123,NaN,NaN,NaN
3134325,21001101111122,NaN,NaN,NaN
3134326,21003102011232,NaN,NaN,NaN
3134327,30012330211323,NaN,NaN,NaN


In [43]:
stime.precipitation_year = stime.precipitation_year.fillna(stime.precipitation_year13)
stime

,quadkey,precipitation_year,approx,precipitation_year13
0,21000132030002,1.154,0.0,NaN
1,21000132012202,1.230,0.0,NaN
2,21000132012000,1.344,0.0,NaN
3,21000132010022,1.406,0.0,NaN
4,21000132212101,1.116,0.0,NaN
...,...,...,...,...
3134324,12200210101123,NaN,NaN,NaN
3134325,21001101111122,NaN,NaN,NaN
3134326,21003102011232,NaN,NaN,NaN
3134327,30012330211323,NaN,NaN,NaN


In [44]:
stime.loc[stime.precipitation_year13.notnull(), 'approx'] = 1

In [45]:
stime[stime.precipitation_year13.notnull()].describe()

,precipitation_year,approx,precipitation_year13
count,392244.000000,392244.0,392244.000000
mean,2.321732,1.0,2.321732
std,2.207667,0.0,2.207667
min,0.000000,1.0,0.000000
25%,0.632000,1.0,0.632000
50%,1.816000,1.0,1.816000
75%,3.344000,1.0,3.344000
max,24.228001,1.0,24.228001


The process is repeated for the quadkey 12 estimates. The approximation is 2.

In [46]:
stime.drop('precipitation_year13', axis = 1, inplace = True)
stime = stime.merge(df.loc[df.precipitation_year12.notnull(),['quadkey', 'precipitation_year12']], on = 'quadkey', how = 'left')
stime

,quadkey,precipitation_year,approx,precipitation_year12
0,21000132030002,1.154,0.0,NaN
1,21000132012202,1.230,0.0,NaN
2,21000132012000,1.344,0.0,NaN
3,21000132010022,1.406,0.0,NaN
4,21000132212101,1.116,0.0,NaN
...,...,...,...,...
3134324,12200210101123,NaN,NaN,NaN
3134325,21001101111122,NaN,NaN,0.644000
3134326,21003102011232,NaN,NaN,NaN
3134327,30012330211323,NaN,NaN,4.432000


In [33]:
stime[(stime.precipitation_year12.notnull()) & (stime.precipitation_year.isnull())].describe()

,precipitation_year,approx,precipitation_year12
count,0.0,0.0,1.561255e+06
mean,NaN,NaN,2.326189e+00
std,NaN,NaN,2.217152e+00
min,NaN,NaN,0.000000e+00
25%,NaN,NaN,6.360000e-01
50%,NaN,NaN,1.816000e+00
75%,NaN,NaN,3.354000e+00
max,NaN,NaN,2.754200e+01


In [47]:
stime.loc[(stime.precipitation_year12.notnull()) & (stime.precipitation_year.isnull()), 'approx'] = 2

In [48]:
stime.precipitation_year = stime.precipitation_year.fillna(stime.precipitation_year12)
stime

,quadkey,precipitation_year,approx,precipitation_year12
0,21000132030002,1.154000,0.0,NaN
1,21000132012202,1.230000,0.0,NaN
2,21000132012000,1.344000,0.0,NaN
3,21000132010022,1.406000,0.0,NaN
4,21000132212101,1.116000,0.0,NaN
...,...,...,...,...
3134324,12200210101123,NaN,NaN,NaN
3134325,21001101111122,0.644000,2.0,0.644000
3134326,21003102011232,NaN,NaN,NaN
3134327,30012330211323,4.432000,2.0,4.432000


In [50]:
stime.drop('precipitation_year12', axis = 1, inplace = True)
stime.isnull().sum()

quadkey                     0
precipitation_year    1049965
approx                1049965
dtype: int64

We repeat the process for the quadkey 11 estimates. The approximation is 3.

In [53]:
stime = stime.merge(df.loc[df.precipitation_year11.notnull(),['quadkey', 'precipitation_year11']], on = 'quadkey', how = 'left')
stime

,quadkey,precipitation_year,approx,precipitation_year11
0,21000132030002,1.154000,0.0,NaN
1,21000132012202,1.230000,0.0,NaN
2,21000132012000,1.344000,0.0,NaN
3,21000132010022,1.406000,0.0,NaN
4,21000132212101,1.116000,0.0,NaN
...,...,...,...,...
3134324,12200210101123,NaN,NaN,1.6320
3134325,21001101111122,0.644000,2.0,0.5570
3134326,21003102011232,NaN,NaN,1.6780
3134327,30012330211323,4.432000,2.0,3.3760


In [54]:
stime.loc[(stime.precipitation_year.isnull()) & (stime.precipitation_year11.notnull()), 'approx'] = 3

In [56]:
stime.precipitation_year = stime.precipitation_year.fillna(stime.precipitation_year11)
stime

,quadkey,precipitation_year,approx,precipitation_year11
0,21000132030002,1.154000,0.0,NaN
1,21000132012202,1.230000,0.0,NaN
2,21000132012000,1.344000,0.0,NaN
3,21000132010022,1.406000,0.0,NaN
4,21000132212101,1.116000,0.0,NaN
...,...,...,...,...
3134324,12200210101123,1.632000,3.0,1.6320
3134325,21001101111122,0.644000,2.0,0.5570
3134326,21003102011232,1.678000,3.0,1.6780
3134327,30012330211323,4.432000,2.0,3.3760


In [57]:
stime.drop('precipitation_year11', axis = 1, inplace = True)
stime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3134329 entries, 0 to 3134328
Data columns (total 3 columns):
 #   Column              Dtype  
---  ------              -----  
 0   quadkey             object 
 1   precipitation_year  float64
 2   approx              float64
dtypes: float64(2), object(1)
memory usage: 71.7+ MB


We can see that the majority of the dataset is estimated with an approximation of 2 or 3, which means it took expanding its area by 16 or 64 times to find a value used as estimate.

In [55]:
stime.approx.value_counts()

approx
2.0    1561255
3.0    1049965
1.0     392244
0.0     130865
Name: count, dtype: int64

We add the quadkey with no estimation back.

In [59]:
missing = list(set(list_quads) - set(stime.quadkey.unique()))
len(missing)

704890

In [60]:
704890 / (704890 + 3134328)

0.18360249404956946

In [61]:
agg = pd.DataFrame(missing, columns = ['quadkey'])

stime = pd.concat([stime, agg], ignore_index = True)
stime

,quadkey,precipitation_year,approx
0,21000132030002,1.154,0.0
1,21000132012202,1.230,0.0
2,21000132012000,1.344,0.0
3,21000132010022,1.406,0.0
4,21000132212101,1.116,0.0
...,...,...,...
3839214,3311101012121,NaN,NaN
3839215,3331322011302,NaN,NaN
3839216,3131112322130,NaN,NaN
3839217,3131130213230,NaN,NaN


In [62]:
stime.to_csv(f'{cart}\\precipitation_utili_exploc.csv', index = False)